## 1. SQL


### Brushing up SQL Concepts

Here are the conceps you should brush-up on or learn to have a basic understanding of SQL and complete the assignment:

1. **Basic SQL Queries**  
   - Understanding the `SELECT` statement, filtering with `WHERE`, and limiting results with `LIMIT`.

2. **Sorting and Aggregation**  
   - Using `ORDER BY`, `GROUP BY`, `HAVING` clauses and aggregate functions like `COUNT`, `SUM`, etc.

3. **Joins and Relationships**  
   - Combining data from multiple tables using various joins (e.g., `INNER JOIN`, `LEFT JOIN`, self-joins).

4. **Subqueries and Nested Queries**  
   - Writing queries within queries to filter or compute intermediate results.

5. **Common Table Expressions (CTEs)**  
   - Creating temporary result sets to simplify complex queries.

6. **Window Functions**  
   - Applying functions across a set of rows related to the current row (e.g., `RANK`, `ROW_NUMBER`).

7. **Transactions and Data Manipulation**  
   - Using `UPDATE`, `DELETE`, `INSERT`, and managing transactions with commands like `BEGIN` and `COMMIT`.

8. **String Aggregation**  
   - Techniques to combine multiple row values into a single string (e.g., using `GROUP_CONCAT`).


### Resources
I find the best way to learn is to start with looking up a specific concept and then playing around with it using an actual database (can use the Open Flights DB below). It can be also be helpful to use these interactive SQL learning tools:

- SQL ZOO : https://sqlzoo.net/wiki/SQL_Tutorial
- SQL Tutorial : https://www.w3schools.com/sql/

Here is a website for comprehensive sqlite syntax tutorials: https://www.sqlitetutorial.net/

If you want to build a more foundational knowlegde in SQL and then play with the queries:
- cannot go wrong with freecodecamp: https://www.youtube.com/watch?v=HXV3zeQKqGY

### Setting Up sqlite DB

We are going to be using the open flights database. The following code will download the data and then set up a sqllite DB that will be the basis of the quiz. Feel free to play around with using SQL queries

In [4]:
import sqlite3
import pandas as pd
import requests
import io

# Dictionary containing dataset URLs and column names
url_dict = {
    'airports': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
                 ["Airport ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude",
                  "Altitude", "Timezone", "DST", "Tz database timezone", "Type", "Source"]),

    'airlines': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat',
                 ["Airline ID", "Name", "Alias", "IATA", "ICAO", "Callsign", "Country", "Active"]),

    'routes': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat',
               ["Airline", "Airline ID", "Source airport", "Source airport ID",
                "Destination airport", "Destination airport ID", "Codeshare", "Stops", "Equipment"]),

    'planes': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat',
               ["Name", "IATA code", "ICAO code"])
}

# Create SQLite database connection
conn = sqlite3.connect("openflights.db")
cursor = conn.cursor()

# Function to fetch data using requests
def fetch_data(url, columns):
    response = requests.get(url)  # Ensures SSL verification
    response.raise_for_status()  # Raise an error if request fails
    data = io.StringIO(response.text)  # Convert text response to a file-like object
    df = pd.read_csv(data, names=columns)  # Read CSV
    df.columns = df.columns.str.lower().str.replace(" ", "_")  # Format column names
    return df

# Loop through datasets, fetch and save them to the SQLite database
for db_name, (url, columns) in url_dict.items():
    df = fetch_data(url, columns)
    print(f"Loaded {db_name} dataset:")
    print(df.head())  # Preview first few rows
    df.to_sql(db_name, conn, if_exists='replace', index=False)

# Close the database connection
conn.close()
print("Database has been successfully updated.")

Loaded airports dataset:
   airport_id                                         name          city  \
0           1                               Goroka Airport        Goroka   
1           2                               Madang Airport        Madang   
2           3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3           4                               Nadzab Airport        Nadzab   
4           5  Port Moresby Jacksons International Airport  Port Moresby   

            country iata  icao  latitude   longitude  altitude timezone dst  \
0  Papua New Guinea  GKA  AYGA -6.081690  145.391998      5282       10   U   
1  Papua New Guinea  MAG  AYMD -5.207080  145.789001        20       10   U   
2  Papua New Guinea  HGU  AYMH -5.826790  144.296005      5388       10   U   
3  Papua New Guinea  LAE  AYNZ -6.569803  146.725977       239       10   U   
4  Papua New Guinea  POM  AYPY -9.443380  147.220001       146       10   U   

   tz_database_timezone     type       sour

In [12]:
#use multiline queries to make it easier to read nad build
query = """
SELECT al.name, al.airline_id, COUNT(DISTINCT r.source_airport) AS distinct_source_airports
FROM airlines al
JOIN routes r ON al.airline_id = r.airline_id
WHERE al.active = 'Y'
GROUP BY al.name, al.airline_id
HAVING COUNT(DISTINCT r.source_airport) >= 3
ORDER BY distinct_source_airports DESC
"""

df = pd.read_sql_query(query,conn)
df

,name,airline_id,distinct_source_airports
0,American Airlines,24,429
1,United Airlines,5209,426
2,Air France,137,378
3,KLM Royal Dutch Airlines,3090,360
4,US Airways,5265,348
...,...,...,...
496,Kam Air,3097,3
497,Omni Air International,3781,3
498,PB Air,3835,3
499,Rossiya-Russian Airlines,641,3


### SQL Quiz

*All questions require SQL queries, statements, or transactions. Provide your answers as SQL code.*

1. Retrieve all columns for the first 5 rows from the `airports` table.

``SELECT *
FROM airports
LIMIT 5;``

2. Select the `name`, `city`, and `country` columns from the `airports` table for airports located in the United States.

``SELECT name, city, country
FROM airports
WHERE country = 'United States';``

3. (Slight Challenge) Find all active airlines (where `active = 'Y'`) that operate from at least 3 distinct source airports. Display the airline's name, airline_id, and the count of distinct source airports. Order the results by the count (highest first).

``SELECT al.name, al.airline_id, COUNT(DISTINCT r.source_airport) AS distinct_source_airports
FROM airlines al
JOIN routes r ON al.airline_id = r.airline_id
WHERE al.active = 'Y'
GROUP BY al.name, al.airline_id
HAVING COUNT(DISTINCT r.source_airport) >= 3
ORDER BY distinct_source_airports DESC;``

4. List all airports that serve as a source airport along with their name, city, country, altitude, and the total number of routes departing from that airport. Only include airports with at least one departing route. Order the results by the total number of routes (descending) and then by altitude (descending).

``SELECT a.name, a.city, a.country, a.altitude, COUNT(*) AS total_routes
FROM airports a
JOIN routes r ON a.airport_id = r.source_airport_id
GROUP BY a.airport_id, a.name, a.city, a.country, a.altitude
HAVING COUNT(*) >= 1
ORDER BY total_routes DESC, a.altitude DESC;``

5. List each source airport along with the count of routes departing from it and its rank based on the route count (with the highest count ranked 1). Only include airports with at least 5 routes.

``SELECT source_airport, total_routes,
RANK() OVER (ORDER BY total_routes DESC) AS route_rank
FROM (
SELECT source_airport, COUNT(*) AS total_routes
FROM routes
GROUP BY source_airport
HAVING COUNT(*) >= 5
) AS sub;``

6. (Slight Challenge) List each route's details including the airline name and airline country, the source airport name and city, and the destination airport name and city. Only include routes where both the source and destination airports are located in the same country as the airline.

``SELECT r.*,
al.name AS airline_name,
al.country AS airline_country,
a_src.name AS source_airport_name,
a_src.city AS source_airport_city,
a_dest.name AS destination_airport_name,
a_dest.city AS destination_airport_city
FROM routes r
JOIN airlines al ON r.airline_id = al.airline_id
JOIN airports a_src ON r.source_airport_id = a_src.airport_id
JOIN airports a_dest ON r.destination_airport_id = a_dest.airport_id
WHERE a_src.country = al.country
AND a_dest.country = al.country;``

7. Find all airports that do not appear in the routes table at all (neither as a source nor as a destination). List the airport's name, city, and country.

``SELECT a.name, a.city, a.country
FROM airports a
LEFT JOIN routes r1 ON a.airport_id = r1.source_airport_id
LEFT JOIN routes r2 ON a.airport_id = r2.destination_airport_id
WHERE r1.source_airport_id IS NULL
AND r2.destination_airport_id IS NULL;``

8. List each source airport once along with a comma-separated list of all its distinct destination airports. Only include source airports that have multiple distinct destination airports.

``SELECT source_airport, GROUP_CONCAT(DISTINCT destination_airport) AS destinations
FROM routes
GROUP BY source_airport
HAVING COUNT(DISTINCT destination_airport) > 1;``

9. Update the `active` status to `'N'` for the airline with a specific `airline_id` (e.g., `1234`) in the `airlines` table.

``UPDATE airlines
SET active = 'N'
WHERE airline_id = 1234;``

10. Create a SQL transaction that deletes all routes from the `routes` table where `stops` is greater than 0, commits the transaction, and then verifies that no routes with `stops` greater than 0 remain.

``BEGIN TRANSACTION;
DELETE FROM routes
WHERE stops > 0;
COMMIT;
SELECT *
FROM routes
WHERE stops > 0;``

## 2. Workflow for Assignment Submission

You will have your own fork of the master repository controlled by Pratyush. For each new assignment, update your fork with the latest changes from Pratyush's master and then create a dedicated branch for your assignment. This approach avoids merge conflicts with files such as IPython Notebook files and keeps your work isolated.

### Workflow Overview

1. Fork Pratyush's Repository  
2. Clone Your Fork Locally  
3. Set Up the Upstream Remote  
4. Update Your Local Master with Upstream Changes  
5. Create a New Branch for the Assignment  
6. Work on and Commit Your Assignment  
7. Push Your Assignment Branch to Your Fork

---

### 1. Fork Pratyush’s Repository

- Navigate to Pratyush’s repository.
- Click the **Fork** button.
- This creates your personal copy of the repository on your GitHub account.

---

### 2. Clone Your Fork Locally

Open your terminal and run:

`git clone https://github.com/your-username/data-alchemy-assignments.git`
Please use name your fork "data_alchemy_assignments" as well. 

Then change into the repository directory:

`cd data-alchemy-assignments`
If you cloned Pratyush's repo for assignment 1, please delete that before to avoid a conflict.

---

### 3. Set Up the Upstream Remote

Add Pratyush’s repository as the upstream remote:

`git remote add upstream https://github.com/pratyush-kundu-99/data_alchemy_assignments.git`

Verify your remotes with:

`git remote -v`

You should see two remotes:
- **origin** pointing to your fork  
- **upstream** pointing to Pratyush's repository

---

### 4. Update Your Local Master with Upstream Changes

If you have made any changes in your master branch, you can stash them before pulling. To stash your changes, run:

`git stash`

*Remember, dont push changes to your master as it will create merge conflicts when you try to pull new changes from the upstream (Pratyush's Assignments repo)*


Then, fetch the latest changes from upstream:

`git fetch upstream`

Switch to your master branch:

`git checkout master`

Merge the upstream master into your local master:

`git merge upstream/master`

If you stashed changes earlier and want to reapply them after updating master, run:

`git stash pop`

*Note: Stashing ensures that any local modifications on your master branch do not conflict with the updates you are pulling from upstream.*

---

### 5. Create a New Branch for the Assignment

Create a new branch for the current assignment:

`git checkout -b assignment-week2`

**Please follow this naming convention for your assignment branches : 'assignment-week<week-number>'**

---

### 6. Work on and Commit Your Assignment

Edit the assignment files as needed. When you are ready to save your work, add and commit your changes:

`git add .`  
`git commit -m "Completed week2 assignment: brief description of changes"`

---

### 7. Push Your Assignment Branch to Your Fork

Push your branch to your fork on GitHub:

`git push origin assignment-week2`

---

### Summary Diagram

`Project Leader's Master Repository (upstream) is now Pratyush's Master Repository`  
`         │`  
`         │  (Pratyush pushes new assignments)`  
`         ▼`  
`      Forks (each student's copy)`  
`         │`  
`         ├── Clone locally and set up 'origin' and 'upstream'`  
`         │`  
`         ├── Regularly:`  
`         │    git fetch upstream`  
`         │    git checkout master`  
`         │    (if there are local changes, use git stash before pulling)`  
`         │    git merge upstream/master`  
`         │`  
`         ├── Create a new branch (e.g. week2-assignment)`  
`         │`  
`         ├── Work on the assignment and commit changes`  
`         │`  
`         └── Push the branch to your fork`

---

### Tips

- **Stashing Local Changes:**  
  Always stash any uncommitted changes in your master branch before fetching and merging upstream updates to avoid merge conflicts.  
- **Branch Discipline:**  
  Do not work directly on the master branch. Always create a new branch for each assignment to isolate your work.  


## 3. Additional Tasks

1. How would you put your Build Project experience in your resume?

For my resume, ideally I would like to put The Build Project under my coursework/certificates area, and my final project as my top project for my projects section.

2. List 3-5 datasets that you would choose from for your final project. This is your chance to pick something in a topic that interests you. We will cover how to find datasets in next session.

For my final project, I would be very interested in working on datasets such as automotive tire test results, stock market price history, phishing websites and LinkedIn + outside job postings